# STATIC STUFF 
 
THIS PART IS USED FOR THE SETUP OF THIS NOTEBOOK

Establish connection

In [1]:
from modules.athena_queries import AthenaQueries
q = AthenaQueries()

Test connection|

In [2]:
df = q.athena_query('SELECT * FROM "default"."details" limit 10;')
df

,id,lng,lat,name,county,parish,abs_price,abs_size,title,room_nr,url,partition_0,partition_1,partition_2,partition_3
0,3215996,26.366362362708,59.348214331755,Rakvere,7,1050,62990,68,,3,https://www.kv.ee/3215996,sale,Rakvere,2020-04-05,3
1,3181368,26.3556586,59.3408665,Rakvere,7,1050,39000,65,ARHAILINE KORTER RAKVERE VANALINNAS,3,https://www.kv.ee/3181368,sale,Rakvere,2020-04-05,3
2,3103126,26.3575213,59.3376908,Rakvere,7,1050,37900,61,,3,https://www.kv.ee/3103126,sale,Rakvere,2020-04-05,3
3,3222062,26.381789,59.3461522,Rakvere,7,1050,58000,61,Müüa 3-toaline korter vaikses piirkonnas.,3,https://www.kv.ee/3222062,sale,Rakvere,2020-04-05,3
4,3201307,26.3775858,59.3407584,Rakvere,7,1050,61900,64,"3 toaline korter, Piiri 10, Rakvere, Lääne-Vir...",3,https://www.kv.ee/3201307,sale,Rakvere,2020-04-05,3
5,3057813,26.364859906541,59.353111434916,Rakvere,7,1050,115000,75,MÜÜA ILUS KORTER KESKLINNAS,3,https://www.kv.ee/3057813,sale,Rakvere,2020-04-05,3
6,3210608,26.3624142,59.3447836,Rakvere,7,1050,48000,50,3-toaline korter kesklinnas,3,https://www.kv.ee/3210608,sale,Rakvere,2020-04-05,3
7,3220334,26.3432167,59.353984,Rakvere,7,1050,79000,63,Uus korter!,3,https://www.kv.ee/3220334,sale,Rakvere,2020-04-05,3
8,3222865,26.3714441,59.3436406,Rakvere,7,1050,54900,62,,3,https://www.kv.ee/3222865,sale,Rakvere,2020-04-05,3
9,3186431,26.3733445,59.3400696,Rakvere,7,1050,44000,61,,3,https://www.kv.ee/3186431,sale,Rakvere,2020-04-05,3


In [7]:
%load_ext autoreload
%autoreload

# IDEAS TO CHECK 

+ Select apartments which have occured in my dataset less than their last_occurence - first_occurence. These do occur
+ If I am able to create a grid over the map I can find for certain areas the average rent price for that area. Based on that I can declare some apartments in certain GRID areas as great catches.




## PLOTS TO DO 

0. Visualize the division of how long apartment is active.
    + rent vs sale 
    + price_bucket 
    + city 
    + room_nr
1. Visualize the locations on map
2. Visualize apartments that went fast and that went slow TOP,BOTTOM  5 percent


In [12]:
import plotly.offline as pyo 
import plotly.graph_objs as go

# APARTMENT AVERAGE WAIT DURATION BEFORE DEAL

## RENT

In [14]:

import plotly.offline as pyo 
import plotly.graph_objs as go

def generate_avg_wait_bar(wait_times,title):
    data=[]
    for room_nr in range(1,6):
        df=wait_times[room_nr]
        trace= go.Bar(x=df['city'],y=df['avg_age'],name=f"{room_nr}")
        data.append(trace)
    layout = go.Layout(title=title)
    fig = go.Figure(data=data,layout=layout)
    return fig

In [15]:
rent_wait_times = q.get_avg_wait_time(deal_type="rent")
sale_wait_times = q.get_avg_wait_time(deal_type="sale")

In [16]:
rent_plt = generate_avg_wait_bar(rent_wait_times,"Rent Average Wait")
sale_plt = generate_avg_wait_bar(sale_wait_times,"Sale Average Wait")

In [17]:
pyo.iplot(rent_plt)

In [18]:
pyo.iplot(sale_plt)

Plotting part 

## SALE

In [19]:
res={}

deal_type="sale"
for room_nr in range(1,6):
    query = f"""with t1 as(SELECT city,room_nr,AVG(days_active) as avg_age ,count(*) as cnt FROM "default"."state_view" where days_inactive > 2 and deal_type = '{deal_type}' and room_nr = {room_nr} GROUP BY 1,2)
SELECT * FROM t1 WHERE cnt > 3;"""
    df = pd.read_sql(query,conn)
    res[room_nr]=df 

NameError: name 'pd' is not defined

In [ ]:
data=[]

for room_nr in range(1,6):
    df=res[room_nr]
    trace= go.Bar(x=df['city'],y=df['avg_age'],name=f"{room_nr}")
    data.append(trace)

layout = go.Layout(title="Average duration till deal")
fig = go.Figure(data=data,layout=layout)
pyo.iplot(fig)
